# iCardio XML parser and data assembly

### Imports

In [2]:
import xml.dom.minidom
import pandas as pd
import cobra
import json
import numpy as np

In [13]:
file = xml.dom.minidom.parse("icardio.xml")

### Species names, locations and ids dataframe

In [16]:
model = file.documentElement
species = model.getElementsByTagName("species")
reactions = model.getElementsByTagName("reaction")
reactants = model.getElementsByTagName("listOfReactants")
products = model.getElementsByTagName("listOfProducts")
species_names = []
species_groups = []
species_ids = []
nodes = []
links = []

for element in species:
    species_names.append(element.getAttribute("name"))
    species_groups.append(element.getAttribute("compartment"))
    species_ids.append(element.getAttribute("id"))
     
dataframe = pd.DataFrame({"index": np.arange(1, len(dataframe)+1), "names": species_names, "groups": species_groups, "ids": species_ids})
dataframe.index = np.arange(1, len(dataframe)+1)
dataframe

,index,names,groups,ids
1,1,(10Z)-heptadecenoic acid,c,m00003__91__c__93__
2,2,(10Z)-heptadecenoic acid,r,m00003__91__r__93__
3,3,(10Z)-heptadecenoic acid,s,m00003__91__s__93__
4,4,(10Z)-heptadecenoyl-CoA,c,m00004__91__c__93__
5,5,(10Z)-heptadecenoyl-CoA,m,m00004__91__m__93__
...,...,...,...,...
2886,2886,hyocholoyl-CoA,c,m90185__91__c__93__
2887,2887,protein,c,m90190__91__c__93__
2888,2888,glycogen storage pool,c,m90194__91__c__93__
2889,2889,intracellular lipids,c,m90195__91__c__93__


In [17]:
first_of_type = []
type_counter = []
for x in species_groups:
    if x in type_counter:
        pass
    else:
        type_counter.append(x)

for types in type_counter:
    foo = dataframe.loc[dataframe["groups"] == types, "index"].array[0]
    first_of_type.append(foo)
    
print(type_counter)


['c', 'r', 's', 'm', 'p', 'g', 'l', 'n']


In [18]:
node_names = []
asdf = []
newdataframe = pd.DataFrame({"id": species_ids, "name": species_names})

for reactant in reactants:
    list_of_reactants = reactant.getElementsByTagName("speciesReference")
    nameids = []
    for item in list_of_reactants:
        nameids.append(item.getAttribute("species"))
    asdf.append(nameids)
        
for x in asdf:
    node_names_x = []
    for y in x:
        boo = newdataframe.loc[newdataframe["id"] == y, "name"].array[0]
        node_names_x.append(boo)
    node_names.append(node_names_x)

string_names = []
for name in node_names:
    string_names.append(" + ".join(name))

print(len(string_names))

4200


In [20]:
product_nodes = []
asdfp = []

for reaction in reactions:
    products_in_reaction = reaction.getElementsByTagName("listOfProducts")
    if len(products_in_reaction) != 0:
        for item in products_in_reaction:
            list_of_products = item.getElementsByTagName("speciesReference")
            prod_nameids = []
            for item in list_of_products:
                prod_nameids.append(item.getAttribute("species"))
            asdfp.append(prod_nameids)        
    elif len(products_in_reaction) == 0:
        asdfp.append("")
           
for x in asdfp:
    product_nodes_x = []
    for y in x:
        boop = newdataframe.loc[newdataframe["id"] == y, "name"].array[0]
        product_nodes_x.append(boop)
    product_nodes.append(product_nodes_x)

string_products = []
for name in product_nodes:
    string_products.append(" + ".join(name))

In [21]:
combined_nodes = string_names + string_products
print(len(combined_nodes))

8400


In [22]:
types_list = []
newdataframe2 = pd.DataFrame({"id": species_ids, "type": species_groups})
        
for x in asdf:
    node_types_x = []
    for y in x:
        boo2 = newdataframe2.loc[newdataframe2["id"] == y, "type"].array[0]
        node_types_x.append(boo2)
    types_list.append(node_types_x[0])

In [23]:
prod_types = []
for x in asdfp:
    if x == "":
        prod_types.append("")
    else:
        prod_node_types_x = []
        for y in x:
            boop2 = newdataframe2.loc[newdataframe2["id"] == y, "type"].array[0]
            prod_node_types_x.append(boop2)
        prod_types.append(prod_node_types_x[0])

print(len(prod_types))

4200


In [24]:
combined_types = types_list + prod_types
print(len(combined_types))

8400


In [25]:
reactant_ids = []
product_ids = []

for reactant in reactants:
    reactant_ids.append(reactant.parentNode.getAttribute("name") + "s")

for reaction in reactions:
    products_in_reaction = reaction.getElementsByTagName("listOfProducts")
    if len(products_in_reaction) == 0:
        product_ids.append("")
    else:
        product_ids.append(reaction.getAttribute("name") + "e")

print(len(product_ids))

4200


In [26]:
combined_ids = reactant_ids + product_ids
print(len(combined_ids))

8400


In [6]:
wildtype = cobra.io.read_sbml_model("icardio.xml")
optim_dataset = wildtype.optimize()
optim_dataset

,fluxes,reduced_costs
RCR10001,736.646705,0.000000
RCR10002,0.000000,-0.333330
RCR10004,0.000000,0.000000
RCR10005,70.938840,0.000000
RCR10006,32.741003,0.000000
...,...,...
RCR90207,0.000000,0.000000
RCR90208,0.000000,-0.583327
RCR90209,0.000000,0.000000
RCR90210,-16.571649,0.000000


In [44]:
negative_colours = ["#426f87", "#1f839c", "#0097a9", "#00abad", "#00bea7", "#00d098", "#41e081", "#84ee63", "#c0f93f", "#ffff06"]
#negative colours are blue -> green -> yellow
positive_colours = ["#f6ff00", "#fcea00", "#ffd500", "#ffc000", "#ffa900", "#ff9200", "#ff7a00", "#ff5f00", "#fe4000", "#f80000"]
#positive colours are yellow -> orange -> red


rxn_names = [_.name for _ in wildtype.reactions]
rxn_wt_flux = [_.flux for _ in wildtype.reactions]

colours = []
for flux in rxn_wt_flux:
    if -1000 <= flux <= -900:
        colours.append("#426f87")
    elif -900 < flux <= -800:
        colours.append("#1f839c")
    elif -800 < flux <= -700:
        colours.append("#0097a9")
    elif -700 < flux <= -600:
        colours.append("#00abad")
    elif -600 < flux <= -500:
        colours.append("#00bea7")
    elif -500 < flux <= -400:
        colours.append("#00d098")
    elif -400 < flux <= -300:
        colours.append("#41e081")
    elif -300 < flux <= -200:
        colours.append("#84ee63")
    elif -200 < flux <= -100:
        colours.append("#c0f93f")
    elif -100 < flux <= 0:
        colours.append("#ffffff")
    elif 0 < flux <= 100:
        colours.append("#c0f93f")
    elif 100 < flux <= 200:
        colours.append("#fcea00")
    elif 200 < flux <= 300:
        colours.append("#ffd500")
    elif 300 < flux <= 400:
        colours.append("#ffc000")
    elif 400 < flux <= 500:
        colours.append("#ffa900")
    elif 500 < flux <= 600:
        colours.append("#ff9200")
    elif 600 < flux <= 700:
        colours.append("#ff7a00")
    elif 700 < flux <= 800:
        colours.append("#ff5f00")
    elif 800 < flux <= 900:
        colours.append("#fe4000")
    elif 900 < flux <= 1000:
        colours.append("#f80000")

print(len(colours))

4200


In [45]:
asdf = pd.DataFrame({"fluxes": rxn_wt_flux, "colours": colours})
asdf

,fluxes,colours
0,736.646705,#ff5f00
1,0.000000,#ffffff
2,0.000000,#ffffff
3,70.938840,#c0f93f
4,32.741003,#c0f93f
...,...,...
4195,0.000000,#ffffff
4196,0.000000,#ffffff
4197,0.000000,#ffffff
4198,-16.571649,#ffffff


In [46]:
nodes = []
for index, item in enumerate(combined_ids):
    node = {"name": str(combined_nodes[index]), "group": combined_types[index], "id": item}
    nodes.append(node)

print(len(nodes))

8400


In [47]:
links = []

for index, number in enumerate(reactant_ids):
    link = {"source": number, "target": product_ids[index], "value": colours[index]}
    links.append(link)

print(len(links))

4200


In [48]:
icardio_data = {"nodes": [], "links": []}
icardio_data["nodes"] += nodes
icardio_data["links"] += links

In [49]:
json_string = json.dumps(icardio_data)

with open('icardio_data.json', 'w', encoding='utf-8') as f:
    json.dump(icardio_data, f, ensure_ascii=False, indent=4)